# Mimic-IV Preprocessing
Code originally taken from GRU-ODE-Bayes Preprocessing, simplified and adapted by Neural-Flows, and finally simplified and adapted for Time-IMM Benchmark.

MIMIC-IV-Note dataset was added and preprocessed for Multimodality.

### Data Selection and Inclusion criteria:
- Include only patients in Metavision system
- Retain only patients with a single admission
- Select patients whose admission duration is over 50 days for longer time-series
- Remove patients who are younger than 15 years
- Remove patients without chart events data
- Remove patients that are not included in MIMIC-IV-Note dataset

Please make sure to download these two projects:
- [MIMIC-IV-Note](https://physionet.org/content/mimic-iv-note/2.2/)
- [MIMIC-IV](https://physionet.org/content/mimiciv/3.1/)

So under the folder MIMIC, you would have:
```
mimiciv/3.1/
mimic-iv-note/2.2/
```

All Time Series Processing

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta
import numpy as np
from tqdm import tqdm
import shutil

In [ ]:
fn = "mimiciv/3.1/hosp/admissions.csv.gz"
adm = pd.read_csv(fn)
adm.head()

In [ ]:
# keep only patients present in patients data
patients_df = pd.read_csv("mimiciv/3.1/hosp/patients.csv.gz")
patients_df[["subject_id", "anchor_age"]].head()
adm_dob = pd.merge(patients_df[["subject_id", "anchor_age"]], adm, on="subject_id")

df = adm.groupby("subject_id")["hadm_id"].nunique()
subj_ids = list(df[df == 1].index)
adm_1 = adm_dob.loc[adm_dob["subject_id"].isin(subj_ids)]
print("Number of patients remaining in the dataframe: ")
print(len(adm_1.index))

In [ ]:
# time of stay in ICU
adm_1 = adm_1.copy()
adm_1["admittime"] = pd.to_datetime(adm_1["admittime"], format="%Y-%m-%d %H:%M:%S")
adm_1["dischtime"] = pd.to_datetime(adm_1["dischtime"], format="%Y-%m-%d %H:%M:%S")

adm_1["elapsed_time"] = adm_1["dischtime"] - adm_1["admittime"]
adm_1.head()
adm_1["elapsed_days"] = adm_1["elapsed_time"].dt.days

adm_2 = adm_1.loc[(adm_1["elapsed_days"] > 50)]
print("Number of patients remaining in the dataframe: ")
print(len(adm_2.index))

In [ ]:
# only patients older than 15
adm_2_15 = adm_2.loc[adm_2["anchor_age"] > 15].copy()
print("Number of patients remaining in the dataframe: ")
print(len(adm_2_15.index))

In [ ]:
fn = "mimiciv/3.1/icu/chartevents.csv.gz"
# this file is huge, we need to read in the data in chunks
# chartevents = pd.read_csv(fn, compression='gzip')

# workaround: (takes about 7 min)
ids = np.array([])
for chunk in tqdm(pd.read_csv(fn, chunksize=1000000), desc="Processing chartevents"):
    ids = np.append(ids, chunk["hadm_id"].unique())
    ids = np.unique(ids)

In [ ]:
adm_2_15_chart = adm_2_15.loc[adm_2_15["hadm_id"].isin(ids)].copy()
print("Number of patients remaining in the dataframe: ")
print(len(adm_2_15_chart.index))

In [ ]:
from pathlib import Path

Path("raw").mkdir(exist_ok=True)
adm_2_15_chart.to_csv("raw/admissions_processed.csv")

In [ ]:
adm_3 = adm_2_15_chart.copy()

In [ ]:
# only choose previously selected admission ids (takes about 30 seconds)
inputs = pd.read_csv("mimiciv/3.1/icu/inputevents.csv.gz")
adm_ids = list(adm_3["hadm_id"])
inputs = inputs.loc[inputs["hadm_id"].isin(adm_ids)]
inputs.head()

In [ ]:
# only keep columns of interest
inputs_small = inputs[
    [
        "subject_id",
        "hadm_id",
        "starttime",
        "endtime",
        "itemid",
        "amount",
        "amountuom",
        "rate",
        "rateuom",
        "patientweight",
        "ordercategorydescription",
    ]
]
print("Number of patients remaining in the database: ")
print(inputs_small["subject_id"].nunique())

In [ ]:
# get item ids for inputs
item_id = pd.read_csv("mimiciv/3.1/icu/d_items.csv.gz")
item_id_1 = item_id[["itemid", "label"]]
item_id_1.head()

inputs_small_2 = pd.merge(inputs_small, item_id_1, on="itemid")
inputs_small_2.head()
print("Number of patients remaining in the database: ")
print(inputs_small_2["subject_id"].nunique())

In [ ]:
# For each item, evaluate the number of patients who have been given this item
# Select only the inputs with highest occurence
pat_for_item = inputs_small_2.groupby("label")["subject_id"].nunique()
frequent_labels = pat_for_item.sort_values(ascending=False)[:50]
inputs_small_3 = inputs_small_2.loc[
    inputs_small_2["label"].isin(list(frequent_labels.index))
].copy()

print("Number of patients remaining in the database: ")
print(inputs_small_3["subject_id"].nunique())

In [ ]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    print(inputs_small_3.groupby("label")["amountuom"].value_counts())

In [ ]:
##### Cleaning the Cefazolin (remove the ones that are not in dose unit)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["itemid"] == 225850) & (inputs_small_3["amountuom"] != "dose")
    ].index
).copy()
# Cleaning the Cefepime (remove the non dose)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Cefepime")
        & (inputs_small_3["amountuom"] != "dose")
    ].index
).copy()
# Cleaning the Ceftriaxone (remove the non dose)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Ceftriaxone")
        & (inputs_small_3["amountuom"] != "dose")
    ].index
).copy()
# Cleaning the Ciprofloxacin (remove the non dose)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Ciprofloxacin")
        & (inputs_small_3["amountuom"] != "dose")
    ].index
).copy()
# Cleaning the Famotidine (Pepcid) (remove the non dose)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Famotidine (Pepcid)")
        & (inputs_small_3["amountuom"] != "dose")
    ].index
).copy()
# Cleaning the Fentanyl (Concentrate) (remove the non mg)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Fentanyl (Concentrate)")
        & (inputs_small_3["amountuom"] != "mg")
    ].index
).copy()
inputs_small_3.loc[
    (inputs_small_3["label"] == "Fentanyl (Concentrate)")
    & (inputs_small_3["amountuom"] == "mg"),
    "amount",
] *= 1000
inputs_small_3.loc[
    (inputs_small_3["label"] == "Fentanyl (Concentrate)")
    & (inputs_small_3["amountuom"] == "mg"),
    "amountuom",
] = "mcg"
# Cleaning the Heparin Sodium (Prophylaxis) (remove the non dose)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Heparin Sodium (Prophylaxis)")
        & (inputs_small_3["amountuom"] != "dose")
    ].index
).copy()
# Cleaning the Hydromorphone (Dilaudid) (remove the non mg)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Hydromorphone (Dilaudid)")
        & (inputs_small_3["amountuom"] != "mg")
    ].index
).copy()
# Cleaning the Magnesium Sulfate (remove the non grams)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Magnesium Sulfate")
        & (inputs_small_3["amountuom"] != "grams")
    ].index
).copy()
# Cleaning the Propofol (remove the non mg)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Propofol") & (inputs_small_3["amountuom"] != "mg")
    ].index
).copy()
# Cleaning the Metoprolol (remove the non mg)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Metoprolol")
        & (inputs_small_3["amountuom"] != "mg")
    ].index
).copy()
# Cleaning the Piperacillin/Tazobactam (Zosyn) (remove the non dose)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Piperacillin/Tazobactam (Zosyn)")
        & (inputs_small_3["amountuom"] != "dose")
    ].index
).copy()
# Cleaning the Metronidazole (remove the non dose)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Metronidazole")
        & (inputs_small_3["amountuom"] != "dose")
    ].index
).copy()
# Cleaning the Ranitidine (Prophylaxis)(remove the non dose)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Ranitidine (Prophylaxis)")
        & (inputs_small_3["amountuom"] != "dose")
    ].index
).copy()
# Cleaning the Vancomycin (remove the non dose)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Vancomycin")
        & (inputs_small_3["amountuom"] != "dose")
    ].index
).copy()
# Cleaning the Fentanyl. Put the mg to mcg
inputs_small_3.loc[
    (inputs_small_3["itemid"] == 221744) & (inputs_small_3["amountuom"] == "mg"),
    "amount",
] *= 1000
inputs_small_3.loc[
    (inputs_small_3["itemid"] == 221744) & (inputs_small_3["amountuom"] == "mg"),
    "amountuom",
] = "mcg"
# Cleaning of the Pantoprazole (Protonix)
# divide in two (drug shot or continuous treatment and create a new item id for the continuous version)
inputs_small_3.loc[
    (inputs_small_3["itemid"] == 225910)
    & (inputs_small_3["ordercategorydescription"] == "Continuous Med"),
    "label",
] = "Pantoprazole (Protonix) Continuous"
inputs_small_3.loc[
    (inputs_small_3["itemid"] == 225910)
    & (inputs_small_3["ordercategorydescription"] == "Continuous Med"),
    "itemid",
] = 2217441
# remove the non dose from the drug shot version
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Pantoprazole (Protonix)")
        & (inputs_small_3["amountuom"] != "dose")
    ].index
).copy()

In [ ]:
# Additional Preprocessing for MIMIC 4 items
# Cleaning the Acetaminophen-IV (keep mg)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Acetaminophen-IV")
        & (inputs_small_3["amountuom"] != "mg")
    ].index
).copy()

# Cleaning the D5 1/2NS (keep ml)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "D5 1/2NS") & (inputs_small_3["amountuom"] != "ml")
    ].index
).copy()

# Cleaning the Dexmedetomidine (Precedex) (cast all to mg)
inputs_small_3.loc[
    (inputs_small_3["label"] == "Dexmedetomidine (Precedex)")
    & (inputs_small_3["amountuom"] == "mcg"),
    "amount",
] /= 1000
inputs_small_3.loc[
    (inputs_small_3["label"] == "Dexmedetomidine (Precedex)")
    & (inputs_small_3["amountuom"] == "mcg"),
    "amountuom",
] = "mg"

# Cleaning the LR
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "LR") & (inputs_small_3["amountuom"] != "ml")
    ].index
).copy()

# Cleaning the NaCl 0.9%
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "NaCl 0.9%") & (inputs_small_3["amountuom"] != "ml")
    ].index
).copy()

# Cleaning the OR Crystalloid Intake
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "OR Crystalloid Intake")
        & (inputs_small_3["amountuom"] != "ml")
    ].index
).copy()

# Cleaning the PO Intake
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "PO Intake") & (inputs_small_3["amountuom"] != "ml")
    ].index
).copy()

# Cleaning the Pre-Admission/Non-ICU Intake
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Pre-Admission/Non-ICU Intake")
        & (inputs_small_3["amountuom"] != "ml")
    ].index
).copy()

In [ ]:
# Verify
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    print(inputs_small_3.groupby("label")["amountuom"].value_counts())

In [ ]:
# same thing for inputs given in rates
inputs_small_3.groupby("label")["rateuom"].value_counts()

In [ ]:
# Cleaning of Dextrose 5%  (remove the non mL/hour)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Dextrose 5%")
        & (inputs_small_3["rateuom"] != "mL/hour")
    ].index
).copy()
# Cleaning of Magnesium Sulfate (Bolus)  (remove the non mL/hour)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Magnesium Sulfate (Bolus)")
        & (inputs_small_3["rateuom"] != "mL/hour")
    ].index
).copy()
# Cleaning of NaCl 0.9% (remove the non mL/hour)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "NaCl 0.9%")
        & (inputs_small_3["rateuom"] != "mL/hour")
    ].index
).copy()
# Cleaning of Piggyback (remove the non mL/hour)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Piggyback")
        & (inputs_small_3["rateuom"] != "mL/hour")
    ].index
).copy()
# Cleaning of Packed Red Bllod Cells (remove the non mL/hour)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Packed Red Blood Cells")
        & (inputs_small_3["rateuom"] != "mL/hour")
    ].index
).copy()

# additional cleaning for mimic4
# Cleaning of Acetaminophen-IV
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Acetaminophen-IV")
        & (inputs_small_3["rateuom"] != "mg/min")
    ].index
).copy()

# Cleaning of Fentanyl (Concentrate)
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Fentanyl (Concentrate)")
        & (inputs_small_3["rateuom"] != "mcg/hour")
    ].index
).copy()

# Cleaning of Phenylephrine
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Phenylephrine")
        & (inputs_small_3["rateuom"] != "mcg/kg/min")
    ].index
).copy()

# Cleaning of Sterile Water
inputs_small_3 = inputs_small_3.drop(
    inputs_small_3.loc[
        (inputs_small_3["label"] == "Sterile Water")
        & (inputs_small_3["rateuom"] != "mL/hour")
    ].index
).copy()


# Check if a single unit per drug
inputs_small_3.groupby("label")["rateuom"].value_counts()

We now split the entries which are spread in time.
We chose the duration window for the sampling. here we choose 30 minutes. So every entry which has a rate and with duration larger than 1 hour, we split it into fixed times injections.

In [ ]:
df_temp = inputs_small_3.loc[
    (inputs_small_3["rate"].notnull())
    & (inputs_small_3["rateuom"].str.contains("mg/min"))
].copy()
df_temp["computed_amount"] = df_temp["rate"] * (
    (
        pd.to_datetime(df_temp["endtime"]) - pd.to_datetime(df_temp["starttime"])
    ).dt.total_seconds()
    / 60
)

# Check with a 0.01 tolerance
assert (
    len(df_temp.loc[(abs(df_temp["computed_amount"] - df_temp["amount"]) > 0.01)].index)
    == 0
)  # OK

# Third check the kg/min units
df_temp = inputs_small_3.loc[
    (inputs_small_3["rate"].notnull())
    & (inputs_small_3["rateuom"].str.contains("mcg/kg/min"))
].copy()
df_temp["computed_amount"] = (
    df_temp["rate"]
    * (
        (
            pd.to_datetime(df_temp["endtime"]) - pd.to_datetime(df_temp["starttime"])
        ).dt.total_seconds()
        / 60
    )
    * df_temp["patientweight"]
)

# Check with a 0.01 tolerance
assert (
    len(
        df_temp.loc[
            (abs(df_temp["computed_amount"] / 1000 - df_temp["amount"]) > 0.01)
        ].index
    )
    == 0
)  # OK

In [ ]:
inputs_small_3.head()

In [ ]:
duration_split_hours = 0.5
to_sec_fact = 3600 * duration_split_hours

# split data set in four.

# The first dataframe contains the entries with no rate but with extended duration inputs (over 0.5 hour)
df_temp1 = (
    inputs_small_3.loc[
        (
            (
                pd.to_datetime(inputs_small_3["endtime"])
                - pd.to_datetime(inputs_small_3["starttime"])
            )
            > timedelta(hours=duration_split_hours)
        )
        & (inputs_small_3["rate"].isnull())
    ]
    .copy()
    .reset_index(drop=True)
)
# The second dataframe contains the entries with no rate and low duration entries (<0.5hour)
df_temp2 = (
    inputs_small_3.loc[
        (
            (
                pd.to_datetime(inputs_small_3["endtime"])
                - pd.to_datetime(inputs_small_3["starttime"])
            )
            <= timedelta(hours=duration_split_hours)
        )
        & (inputs_small_3["rate"].isnull())
    ]
    .copy()
    .reset_index(drop=True)
)
# The third dataframe contains the entries with a rate and extended duration inputs (over 0.5 hour)
df_temp3 = (
    inputs_small_3.loc[
        (
            (
                pd.to_datetime(inputs_small_3["endtime"])
                - pd.to_datetime(inputs_small_3["starttime"])
            )
            > timedelta(hours=duration_split_hours)
        )
        & (inputs_small_3["rate"].notnull())
    ]
    .copy()
    .reset_index(drop=True)
)
# The forth dataframe contains the entries with a rate and low duration entries (< 0.5 hour)
df_temp4 = (
    inputs_small_3.loc[
        (
            (
                pd.to_datetime(inputs_small_3["endtime"])
                - pd.to_datetime(inputs_small_3["starttime"])
            )
            <= timedelta(hours=duration_split_hours)
        )
        & (inputs_small_3["rate"].notnull())
    ]
    .copy()
    .reset_index(drop=True)
)

# Check if split is complete
assert len(df_temp1.index) + len(df_temp2.index) + len(df_temp3.index) + len(
    df_temp4.index
) == len(inputs_small_3.index)

In [ ]:
# We then process all of these dfs.
# In the first one, we need to duplicate the entries according to their duration and then divide each entry by the number of duplicates

# We duplicate the rows with the number bins for each injection
df_temp1["Repeat"] = np.ceil(
    (
        pd.to_datetime(df_temp1["endtime"]) - pd.to_datetime(df_temp1["starttime"])
    ).dt.total_seconds()
    / to_sec_fact
).astype(int)
df_new1 = df_temp1.reindex(df_temp1.index.repeat(df_temp1["Repeat"]))

In [ ]:
# We then create the admninistration time as a shifted version of the STARTTIME.
df_new1["charttime"] = df_new1.groupby(level=0)["starttime"].transform(
    lambda x: pd.date_range(
        start=x.iat[0], freq=str(60 * duration_split_hours) + "min", periods=len(x)
    )
)
# We divide each entry by the number of repeats
df_new1["amount"] = df_new1["amount"] / df_new1["Repeat"]

In [ ]:
# In the third one, we do the same
# We duplicate the rows with the number bins for each injection
df_temp3["Repeat"] = np.ceil(
    (
        pd.to_datetime(df_temp3["endtime"]) - pd.to_datetime(df_temp3["starttime"])
    ).dt.total_seconds()
    / to_sec_fact
).astype(int)
df_new3 = df_temp3.reindex(df_temp3.index.repeat(df_temp3["Repeat"]))
# We then create the admninistration time as a shifted version of the STARTTIME.

In [ ]:
df_new3["charttime"] = df_new3.groupby(level=0)["starttime"].transform(
    lambda x: pd.date_range(
        start=x.iat[0], freq=str(60 * duration_split_hours) + "min", periods=len(x)
    )
)
# We divide each entry by the number of repeats
df_new3["amount"] = df_new3["amount"] / df_new3["Repeat"]

df_temp2["charttime"] = df_temp2["starttime"]
df_temp4["charttime"] = df_temp4["starttime"]

In [ ]:
# Eventually, we merge all 4splits into one.
inputs_small_4 = pd.concat([df_new1, df_temp2, df_new3, df_temp4])
# The result is a dataset with discrete inputs for each treatment.

In [ ]:
inputs_small_4.to_csv("raw/inputs_processed.csv")
inputs_small_4["hadm_id"].nunique()

In [ ]:
pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", 300)

In [ ]:
adm = adm_3.copy()

In [ ]:
# Takes about 4 min
df = pd.DataFrame()
for chunk in pd.read_csv("mimiciv/3.1/hosp/labevents.csv.gz", chunksize=500000):
    adm_ids = list(adm["hadm_id"])
    chunk = chunk.loc[chunk["hadm_id"].isin(adm_ids)]
    df = pd.concat(
        [df, chunk[["subject_id", "hadm_id", "charttime", "valuenum", "itemid"]]]
    )

In [ ]:
# only choose previously selected admission ids.
print("Number of patients remaining in the database: ")
print(df["subject_id"].nunique())

In [ ]:
# get item ids
item_id = pd.read_csv("mimiciv/3.1/hosp/d_labitems.csv.gz")
item_id_1 = item_id[["itemid", "label"]]
item_id_1.head()

In [ ]:
# get names of administered items
lab2 = pd.merge(df, item_id_1, on="itemid")
lab2.head()
print("Number of patients remaining in the database: ")
print(lab2["subject_id"].nunique())

In [ ]:
lab2

In [ ]:
# get only top 150 most used tests
n_best = 150
pat_for_item = lab2.groupby("label")["subject_id"].nunique()
frequent_labels = pat_for_item.sort_values(ascending=False)[:n_best]
lab3 = lab2.loc[lab2["label"].isin(list(frequent_labels.index))].copy()

print("Number of patients remaining in the database: ")
print(lab3["subject_id"].nunique())

In [ ]:
lab3

In [ ]:
# only select the subset that was used in the paper (only missing is INR(PT))
subset = [
    "Albumin",
    "Alanine Aminotransferase (ALT)",
    "Alkaline Phosphatase",
    "Anion Gap",
    "Asparate Aminotransferase (AST)",
    "Base Excess",
    "Basophils",
    "Bicarbonate",
    "Bilirubin, Total",
    "Calcium, Total",
    "Calculated Total CO2",
    "Chloride",
    "Creatinine",
    "Eosinophils",
    "Glucose",
    "Hematocrit",
    "Hemoglobin",
    "Lactate",
    "Lymphocytes",
    "MCH",
    "MCV",
    "Magnesium",
    "Monocytes",
    "Neutrophils",
    "PT",
    "PTT",
    "Phosphate",
    "Platelet Count",
    "Potassium",
    "RDW",
    "Red Blood Cells",
    "Sodium",
    "Specific Gravity",
    "Urea Nitrogen",
    "White Blood Cells",
    "pCO2",
    "pH",
    "pO2",
]

lab4 = lab3.loc[lab3["label"].isin(subset)].copy()

In [ ]:
lab4

In [ ]:
lab4.to_csv("raw/lab_processed.csv")

In [ ]:
# Takes about 1 min
# only choose previously selected admission ids
presc = pd.read_csv("mimiciv/3.1/hosp/prescriptions.csv.gz")
adm_ids = list(adm["hadm_id"])
presc = presc.loc[presc["hadm_id"].isin(adm_ids)]

print("Number of patients remaining in the database: ")
print(presc["subject_id"].nunique())
presc.tail()

In [ ]:
# Select entries whose drug name is in the list from the paper.
drugs_list = [
    "Acetaminophen",
    "Aspirin",
    "Bisacodyl",
    "Insulin",
    "Heparin",
    "Docusate Sodium",
    "D5W",
    "Humulin-R Insulin",
    "Potassium Chloride",
    "Magnesium Sulfate",
    "Metoprolol Tartrate",
    "Sodium Chloride 0.9%  Flush",
    "Pantoprazole",
]
presc2 = presc.loc[presc["drug"].isin(drugs_list)]

print("Number of patients remaining in the database: ")
print(presc2["subject_id"].nunique())

In [ ]:
print(presc2.groupby("drug")["dose_unit_rx"].value_counts())

In [ ]:
# Units correction
presc2 = presc2.drop(presc2.loc[presc2["dose_unit_rx"].isnull()].index).copy()
presc2 = presc2.drop(
    presc2.loc[
        (presc2["drug"] == "Acetaminophen") & (presc2["dose_unit_rx"] != "mg")
    ].index
).copy()
presc2.loc[
    (presc2["drug"] == "D5W") & (presc2["dose_unit_rx"] == "ml"), "dose_unit_rx"
] = "mL"
presc2 = presc2.drop(
    presc2.loc[(presc2["drug"] == "D5W") & (presc2["dose_unit_rx"] != "mL")].index
).copy()
presc2 = presc2.drop(
    presc2.loc[(presc2["drug"] == "Heparin") & (presc2["dose_unit_rx"] != "UNIT")].index
).copy()
presc2 = presc2.drop(
    presc2.loc[(presc2["drug"] == "Insulin") & (presc2["dose_unit_rx"] != "UNIT")].index
).copy()
presc2 = presc2.drop(
    presc2.loc[
        (presc2["drug"] == "Magnesium Sulfate") & (presc2["dose_unit_rx"] != "gm")
    ].index
).copy()
presc2 = presc2.drop(
    presc2.loc[
        (presc2["drug"] == "Potassium Chloride") & (presc2["dose_unit_rx"] != "mEq")
    ].index
).copy()
presc2.loc[
    (presc2["drug"] == "Sodium Chloride 0.9%  Flush")
    & (presc2["dose_unit_rx"] == "ml"),
    "dose_unit_rx",
] = "mL"
presc2 = presc2.drop(
    presc2.loc[(presc2["drug"] == "Bisacodyl") & (presc2["dose_unit_rx"] != "mg")].index
).copy()
presc2 = presc2.drop(
    presc2.loc[
        (presc2["drug"] == "Pantoprazole") & (presc2["dose_unit_rx"] != "mg")
    ].index
).copy()
print(presc2.groupby("drug")["dose_unit_rx"].value_counts())

In [ ]:
# To avoid confounding labels with labels from other tables, we add "drug" to the name
presc2["charttime"] = pd.to_datetime(presc2["starttime"], format="%Y-%m-%d %H:%M:%S")
presc2["drug"] = presc2["drug"] + " Drug"

In [ ]:
presc2.to_csv("raw/prescriptions_processed.csv")

In [ ]:
outputs = pd.read_csv("mimiciv/3.1/icu/outputevents.csv.gz")
outputs.tail()

In [ ]:
# only choose previously selected admission ids
adm_ids = list(adm["hadm_id"])
outputs = outputs.loc[outputs["hadm_id"].isin(adm_ids)]

print("Number of patients remaining in the database: ")
print(outputs["subject_id"].nunique())

In [ ]:
# get item names
item_id = pd.read_csv("mimiciv/3.1/icu/d_items.csv.gz")
item_id_1 = item_id[["itemid", "label"]]
item_id_1.head()

outputs_2 = pd.merge(outputs, item_id_1, on="itemid")
outputs_2.head()
print("Number of patients remaining in the database: ")
print(outputs_2["subject_id"].nunique())

In [ ]:
# take only the n most used items
n_best = 15
pat_for_item = outputs_2.groupby("label")["subject_id"].nunique()
frequent_labels = pat_for_item.sort_values(ascending=False)[:n_best]
outputs_3 = outputs_2.loc[outputs_2["label"].isin(list(frequent_labels.index))].copy()

print("Number of patients remaining in the database: ")
print(outputs_3["subject_id"].nunique())
print("Number of datapoints remaining in the database: ")
print(len(outputs_3.index))

print(frequent_labels)

In [ ]:
outputs_label_list = [
    "Foley",
    "Void",
    "OR Urine",
    "Chest Tube",
    "Oral Gastric",
    "Pre-Admission",
    "TF Residual",
    "OR EBL",
    "Emesis",
    "Nasogastric",
    "Stool",
    "Jackson Pratt",
    "TF Residual Output",
    "Fecal Bag",
    "Straight Cath",
]
outputs_bis = outputs_2.loc[outputs_2["label"].isin(outputs_label_list)].copy()

print("Number of patients remaining in the database: ")
print(outputs_bis["subject_id"].nunique())
print("Number of datapoints remaining in the database: ")
print(len(outputs_bis.index))

outputs_3 = outputs_bis.copy()

In [ ]:
# Verification that all input labels have the same amounts units
outputs_3.groupby("label")["valueuom"].value_counts()

In [ ]:
outputs_3.to_csv("raw/outputs_processed.csv")

In [ ]:
lab_df = pd.read_csv("raw/lab_processed.csv")[
    ["subject_id", "hadm_id", "charttime", "valuenum", "label"]
]
inputs_df = pd.read_csv("raw/inputs_processed.csv")[
    ["subject_id", "hadm_id", "charttime", "amount", "label"]
]
outputs_df = pd.read_csv("raw/outputs_processed.csv")[
    ["subject_id", "hadm_id", "charttime", "value", "label"]
]
presc_df = pd.read_csv("raw/prescriptions_processed.csv")[
    ["subject_id", "hadm_id", "charttime", "dose_val_rx", "drug"]
]

In [ ]:
# Change the name of amount. Valuenum for every table
inputs_df["valuenum"] = inputs_df["amount"]
inputs_df = inputs_df.drop(columns=["amount"]).copy()

In [ ]:
outputs_df["valuenum"] = outputs_df["value"]
outputs_df = outputs_df.drop(columns=["value"]).copy()

In [ ]:
presc_df["valuenum"] = presc_df["dose_val_rx"]
presc_df = presc_df.drop(columns=["dose_val_rx"]).copy()
presc_df["label"] = presc_df["drug"]
presc_df = presc_df.drop(columns=["drug"]).copy()
# Drop rows with non-numeric values
presc_df = presc_df.drop(
    presc_df[presc_df["valuenum"].str.contains("-", na=False)].index
)
presc_df["valuenum"] = (
    presc_df["valuenum"].astype(str).str.replace(",", "", regex=False)
)

In [ ]:
# Tag to distinguish between lab and inputs events
inputs_df["Origin"] = "Inputs"
lab_df["Origin"] = "Lab"
outputs_df["Origin"] = "Outputs"
presc_df["Origin"] = "Prescriptions"

In [ ]:
# merge both dfs.
merged_df1 = (pd.concat([inputs_df, lab_df])).reset_index(drop=True)
merged_df2 = (pd.concat([merged_df1, outputs_df])).reset_index(drop=True)
merged_df = (pd.concat([merged_df2, presc_df])).reset_index(drop=True)
assert merged_df["label"].nunique() == (
    inputs_df["label"].nunique()
    + lab_df["label"].nunique()
    + outputs_df["label"].nunique()
    + presc_df["label"].nunique()
)

In [ ]:
merged_df["charttime"] = pd.to_datetime(
    merged_df["charttime"], format="%Y-%m-%d %H:%M:%S"
)

In [ ]:
# set the timestamp as the time delta between the first chart time for each admission
merged_df["charttime"] = pd.to_datetime(
    merged_df["charttime"], format="%Y-%m-%d %H:%M:%S"
)
ref_time = merged_df.groupby("hadm_id")["charttime"].min()
merged_df_1 = pd.merge(
    ref_time.to_frame(name="ref_time"), merged_df, left_index=True, right_on="hadm_id"
)
merged_df_1["time_stamp"] = merged_df_1["charttime"] - merged_df_1["ref_time"]
assert len(merged_df_1.loc[merged_df_1["time_stamp"] < timedelta(hours=0)].index) == 0

In [ ]:
# Create a label code (int) for the labels.
label_dict = dict(
    zip(
        list(merged_df_1["label"].unique()),
        range(len(list(merged_df_1["label"].unique()))),
    )
)
merged_df_1["label_code"] = merged_df_1["label"].map(label_dict)

merged_df_short = merged_df_1[
    ["hadm_id", "valuenum", "time_stamp", "label_code", "Origin"]
]

In [ ]:
label_dict_df = pd.Series(merged_df_1["label"].unique()).reset_index()
label_dict_df.columns = ["index", "label"]
label_dict_df["label_code"] = label_dict_df["label"].map(label_dict)
label_dict_df.drop(columns=["index"], inplace=True)
label_dict_df.to_csv("raw/label_dict.csv")

In [ ]:
merged_df_short["valuenum"] = merged_df_short["valuenum"].astype(float)

In [ ]:
# drop columns that are not needed for final dataset
merged_df_short.drop(["Origin"], axis=1, inplace=True)
merged_df_short.dropna(inplace=True)
complete_df = merged_df_short

In [ ]:
# create value- and mask- columns and fill with data
labels = complete_df["label_code"].unique()
value_columns = []
mask_columns = []
for num in labels:
    name = "Value_label_" + str(num)
    name2 = "Mask_label_" + str(num)
    value_columns.append(name)
    mask_columns.append(name2)
    complete_df[name] = 0
    complete_df[name2] = 0
    complete_df[name] = complete_df[name].astype(float)

In [ ]:
# Takes about 2 min
for index, row in complete_df.iterrows():
    name = "Value_label_" + str(row["label_code"])
    name2 = "Mask_label_" + str(row["label_code"])
    complete_df.at[index, name] = row["valuenum"]
    complete_df.at[index, name2] = 1

In [ ]:
# drop all unneccesary columns and do sanity check
complete_df.drop(["valuenum", "label_code"], axis=1, inplace=True)
# merge duplicate rows using hadim_id and time_stamp as keys
complete_df = complete_df.groupby(["hadm_id", "time_stamp"], as_index=False).max()
for x in mask_columns:
    assert len(complete_df.loc[complete_df[x] > 1]) == 0

In [ ]:
# Set Value_label_X to NaN where Mask_label_X is 0
for i in range(109):
    value_col = f"Value_label_{i}"
    mask_col = f"Mask_label_{i}"
    if value_col in complete_df.columns and mask_col in complete_df.columns:
        # Set Value_label_X to NaN where Mask_label_X is 0
        complete_df.loc[complete_df[mask_col] == 0, value_col] = np.nan
# drop rows with missing values
# Drop all Mask_label columns
mask_cols = [col for col in complete_df.columns if col.startswith("Mask_label_")]
complete_df = complete_df.drop(columns=mask_cols)

In [ ]:
hadm_ids = complete_df["hadm_id"].values

Text Feature Preprocessing

In [ ]:
# Takes about 30 seconds
rad_df = pd.DataFrame()
for chunk in pd.read_csv("mimic-iv-note/2.2/note/radiology.csv.gz", chunksize=500000):
    chunk = chunk.loc[chunk["hadm_id"].isin(hadm_ids)]
    rad_df = pd.concat([rad_df, chunk])
rad_df = rad_df.reset_index(drop=True)

In [ ]:
rad_df["charttime"] = pd.to_datetime(rad_df["charttime"], format="%Y-%m-%d %H:%M:%S")
rad_df_1 = pd.merge(
    ref_time.to_frame(name="ref_time"), rad_df, left_index=True, right_on="hadm_id"
)
rad_df_1["time_stamp"] = rad_df_1["charttime"] - rad_df_1["ref_time"]
rad_df_1 = rad_df_1[["hadm_id", "time_stamp", "text"]]

Saving

In [ ]:
# Use only hadm_ids that are in both complete_df (time series) and rad_df_1 (text)
save_hadm_ids = np.unique(rad_df_1["hadm_id"].values)

In [ ]:
folder_path = "raw/"

In [ ]:
# Change time_stamp from day time to '2000-01-01 00:00:00' format, where all time stamps start from 2000-01-01
base_datetime = pd.Timestamp("2000-01-01 00:00:00")
complete_df["time_stamp"] = pd.to_timedelta(complete_df["time_stamp"])
rad_df_1["time_stamp"] = pd.to_timedelta(rad_df_1["time_stamp"])
complete_df["time_stamp"] = complete_df["time_stamp"] + base_datetime
rad_df_1["time_stamp"] = rad_df_1["time_stamp"] + base_datetime

In [ ]:
grouped_by_hadm_id = rad_df_1.groupby("hadm_id")
for hadm_id_float, group_df in grouped_by_hadm_id:
    if hadm_id_float in save_hadm_ids:
        hadm_id_int = int(hadm_id_float)
        folder_name = str(hadm_id_int)
        os.makedirs(folder_path + folder_name, exist_ok=True)
        file_path = folder_path + folder_name + "/text.csv"
        group_df.to_csv(file_path, index=False)

In [ ]:
ts_grouped_by_hadm_id = complete_df.groupby("hadm_id")
for hadm_id_float, group_df in ts_grouped_by_hadm_id:
    if hadm_id_float in save_hadm_ids:
        hadm_id_int = int(hadm_id_float)
        folder_name = str(hadm_id_int)
        os.makedirs(folder_path + folder_name, exist_ok=True)
        file_path = folder_path + folder_name + "/time_series.csv"
        group_df.to_csv(file_path, index=False)

In [ ]:
# Define the root directory
root_dir = "raw"

# Dictionary to store row counts and missing rate
entity_stats = {}

# Traverse each subfolder (entity_id)
for entity_id in os.listdir(root_dir):
    entity_path = os.path.join(root_dir, entity_id)
    time_series_path = os.path.join(entity_path, "time_series.csv")

    if os.path.isdir(entity_path) and os.path.exists(time_series_path):
        try:
            # Load time_series.csv
            df_ts = pd.read_csv(time_series_path)
            time_series_count = len(df_ts)

            # Calculate missing rate: total NaNs / total cells
            total_cells = df_ts.shape[0] * df_ts.shape[1]
            missing_cells = df_ts.isna().sum().sum()
            missing_rate = missing_cells / total_cells if total_cells > 0 else None

            entity_stats[entity_id] = {
                "time_series_rows": time_series_count,
                "missing_rate": missing_rate,
            }
        except Exception as e:
            print(f"Error reading {time_series_path}: {e}")

# Sort by time_series row count and take top 20
top_20_entities = sorted(
    entity_stats.items(), key=lambda x: x[1]["time_series_rows"], reverse=True
)[:20]

# Now, count rows in text.csv and add it (using pandas)
for entity_id, stats in top_20_entities:
    text_path = os.path.join(root_dir, entity_id, "text.csv")
    try:
        df_text = pd.read_csv(text_path)
        text_count = len(df_text)
        stats["text_rows"] = text_count
    except Exception as e:
        print(f"Error reading {text_path}: {e}")
        stats["text_rows"] = None

# Prepare DataFrame
top_20_df = pd.DataFrame(
    [
        (eid, stats["time_series_rows"], stats["text_rows"], stats["missing_rate"])
        for eid, stats in top_20_entities
    ],
    columns=["entity_id", "time_series_rows", "text_rows", "missing_rate"],
)

entity_list = top_20_df["entity_id"].tolist()
entity_list

In [ ]:
important_feature_names = [
    "Dextrose 5%",
    "Sterile Water",
    "Fentanyl",
    "Heparin Sodium",
    "Solution",
    "Propofol",
    "Phenylephrine",
    "Foley",
    "Norepinephrine",
    "Midazolam (Versed)",
    "pH",
    "Base Excess",
    "Calculated Total CO2",
    "pCO2",
    "pO2",
    "Glucose",
    "Sodium",
    "Bicarbonate",
    "Chloride",
    "Urea Nitrogen",
    "Creatinine",
    "Potassium",
    "Vasopressin",
    "Anion Gap",
    "Magnesium",
    "Calcium, Total",
    "Phosphate",
    "Gastric Meds",
    "Insulin - Regular",
    "Piggyback",
]
important_features = [
    "Value_label_" + str(label_dict[label])
    for label in important_feature_names
    if label in label_dict
]
important_features

In [ ]:
def extract_data(entity_list, source_root="raw", target_root="processed"):
    """
    Copies folders of selected entities from source_root to target_root.

    Parameters:
        entity_list (list): List of entity IDs (integers or strings).
        source_root (str): Path to the source root directory (e.g., 'raw').
        target_root (str): Path to the target root directory (e.g., 'processed').
    """
    os.makedirs(target_root, exist_ok=True)

    for entity_id in entity_list:
        src_path = os.path.join(source_root, str(entity_id))
        dst_path = os.path.join(target_root, str(entity_id))

        if os.path.exists(src_path) and os.path.isdir(src_path):
            try:
                shutil.copytree(src_path, dst_path, dirs_exist_ok=True)
                print(f"Copied {entity_id} to {target_root}")
            except Exception as e:
                print(f"Failed to copy {entity_id}: {e}")
        else:
            print(f"Source folder not found for entity {entity_id}")


def process_csv_files(entity_list, root_dir="processed"):
    """
    - For text.csv: renames columns and sorts by date_time, keeps all columns.
    - For time_series.csv: renames columns, filters to important_features + date_time + record_id, sorts and saves.
    """
    for entity_id in entity_list:
        entity_dir = os.path.join(root_dir, entity_id)

        # Process text.csv
        text_path = os.path.join(entity_dir, "text.csv")
        if os.path.exists(text_path):
            df_text = pd.read_csv(text_path)
            df_text.rename(
                columns={"time_stamp": "date_time", "hadm_id": "record_id"},
                inplace=True,
            )
            if "date_time" in df_text.columns:
                df_text.sort_values(by="date_time", inplace=True)
            df_text.to_csv(text_path, index=False)

        # Process time_series.csv
        ts_path = os.path.join(entity_dir, "time_series.csv")
        if os.path.exists(ts_path):
            df_ts = pd.read_csv(ts_path)
            df_ts.rename(
                columns={"time_stamp": "date_time", "hadm_id": "record_id"},
                inplace=True,
            )
            # Determine columns to keep
            cols_to_keep = []
            if "date_time" in df_ts.columns:
                cols_to_keep.append("date_time")
            if "record_id" in df_ts.columns:
                cols_to_keep.append("record_id")
            cols_to_keep += [col for col in important_features if col in df_ts.columns]
            df_ts = df_ts[cols_to_keep]
            if "date_time" in df_ts.columns:
                df_ts.sort_values(by="date_time", inplace=True)
            df_ts.to_csv(ts_path, index=False)


extract_data(entity_list)
process_csv_files(entity_list)